In [2]:
# _____________ read from SQLite and write to Azure
# connect to current source SQLite
import sqlite3
import pandas as pd
conn = sqlite3.connect(r"D:\NMT\OneDrive\Viettravel Airline\Database\VTA_RM.db")
c = conn.cursor()

In [3]:
payment_detail = pd.read_sql_query('SELECT * FROM payment_detail', conn)
payment_detail['modified_time'] = pd.to_datetime(payment_detail['modified_time'])
payment_detail['BOOK_DATE_GMT'] = pd.to_datetime(payment_detail['BOOK_DATE_GMT'])
payment_detail['BOOK_DATE_LCL'] = pd.to_datetime(payment_detail['BOOK_DATE_LCL'])
payment_detail['LAST_MODIFIED_GMT'] = pd.to_datetime(payment_detail['LAST_MODIFIED_GMT'])
payment_detail['LAST_MODIFIED_LCL'] = pd.to_datetime(payment_detail['LAST_MODIFIED_LCL'])
payment_detail['DATE_PAID_GMT'] = pd.to_datetime(payment_detail['DATE_PAID_GMT'])
payment_detail['DATE_PAID_LCL'] = pd.to_datetime(payment_detail['DATE_PAID_LCL'])

In [4]:
payment_detail.iloc[:, 30:40].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378609 entries, 0 to 378608
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   VCHR_CREATED_GMT  455 non-null     object
 1   VCHR_CREATED_LCL  455 non-null     object
 2   RES_CURRENCY      378531 non-null  object
 3   CURRENCY_PAID     378531 non-null  object
 4   AMOUNT_PAID       378531 non-null  object
 5   BASE_CURRENCY     378531 non-null  object
 6   BASE_AMOUNT       378531 non-null  object
 7   RPT_CURRENCY      378531 non-null  object
 8   RPT_AMOUNT        378531 non-null  object
 9   file_name         378609 non-null  object
dtypes: object(10)
memory usage: 28.9+ MB


In [5]:
payment_detail.iloc[:, 30:40].head()

,VCHR_CREATED_GMT,VCHR_CREATED_LCL,RES_CURRENCY,CURRENCY_PAID,AMOUNT_PAID,BASE_CURRENCY,BASE_AMOUNT,RPT_CURRENCY,RPT_AMOUNT,file_name
0,None,None,VND,VND,"664,630.00",VND,"664,630.00",VND,"664,630.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...
1,None,None,VND,VND,"-664,630.00",VND,"-664,630.00",VND,"-664,630.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...
2,None,None,VND,VND,"-15,186,000.00",VND,"-15,186,000.00",VND,"-15,186,000.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...
3,None,None,VND,VND,"-1,089,000.00",VND,"-1,089,000.00",VND,"-1,089,000.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...
4,None,None,VND,VND,"-423,500.00",VND,"-423,500.00",VND,"-423,500.00",ACCOUNTING_003_PAYMENT_DETAIL - 20211115 - 202...


In [6]:
a = payment_detail.iloc[:, 30:40]
a['AMOUNT_PAID'].astype('float64')

ValueError: could not convert string to float: '664,630.00'

In [35]:
payment_detail[payment_detail['AMOUNT_PAID'].isna()]

,CONFIRMATION_NUM,BOOKING_TYPE,RES_STATUS,RES_PAYMENT_ID,BOOK_DATE_GMT,BOOK_DATE_LCL,LAST_MODIFIED_GMT,LAST_MODIFIED_LCL,BOOKING_AGENT,CRS_CODE,...,VCHR_CREATED_LCL,RES_CURRENCY,CURRENCY_PAID,AMOUNT_PAID,BASE_CURRENCY,BASE_AMOUNT,RPT_CURRENCY,RPT_AMOUNT,file_name,modified_time
3190,None,None,NaN,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,ACCOUNTING_003_PAYMENT_DETAIL - 20211213 - 202...,2022-06-27 15:59:47.000000
6440,None,None,NaN,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,ACCOUNTING_003_PAYMENT_DETAIL - 20211220 - 202...,2022-06-27 15:59:13.000000
9660,None,None,NaN,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,ACCOUNTING_003_PAYMENT_DETAIL - 20211227 - 202...,2022-06-27 15:58:27.000000
12420,None,None,NaN,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,ACCOUNTING_003_PAYMENT_DETAIL - 20220103 - 202...,2022-06-27 15:57:49.000000
17796,None,None,NaN,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,ACCOUNTING_003_PAYMENT_DETAIL - 20220110 - 202...,2022-06-27 15:57:14.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368312,None,None,NaN,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,ACCOUNTING_003_PAYMENT_DETAIL - 20220913.csv,2022-09-14 09:03:04.837961
370146,None,None,NaN,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,ACCOUNTING_003_PAYMENT_DETAIL - 20220914.csv,2022-09-15 10:20:48.373500
372038,None,None,NaN,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,ACCOUNTING_003_PAYMENT_DETAIL - 20220915.csv,2022-09-16 09:11:18.735503
376635,None,None,NaN,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,ACCOUNTING_003_PAYMENT_DETAIL - 20220916 - 202...,2022-09-19 09:25:29.029452


In [8]:
payment_detail_files_dir = r'C:\Users\admin\Desktop\VTA\ACCOUNTING_003_PAYMENT_DETAIL - 20220915.csv'
df = pd.read_csv(payment_detail_files_dir, skiprows=2)

In [20]:
df.dropna(how='all')

,CONFIRMATION_NUM,BOOKING_TYPE,RES_STATUS,RES_PAYMENT_ID,BOOK_DATE_GMT,BOOK_DATE_LCL,LAST_MODIFIED_GMT,LAST_MODIFIED_LCL,BOOKING_AGENT,CRS_CODE,...,VOUCHER_NUM_FULL,VCHR_CREATED_GMT,VCHR_CREATED_LCL,RES_CURRENCY,CURRENCY_PAID,AMOUNT_PAID,BASE_CURRENCY,BASE_AMOUNT,RPT_CURRENCY,RPT_AMOUNT
0,58E677,TPAPI,1.0,1344484.0,2022-09-14 16:56:20,2022-09-14 23:56:20,2022-09-14 17:01:40,2022-09-15 00:01:40,tvlkapi03,NaN,...,NaN,NaN,NaN,VND,VND,614640.0,VND,614640.0,VND,614640.0
1,LLEGCX,EXT_WEB,0.0,1344471.0,2022-09-14 17:03:23,2022-09-15 00:03:23,2022-09-15 05:03:27,2022-09-15 12:03:27,ezygtwy,NaN,...,NaN,NaN,NaN,VND,VND,1241040.0,VND,1241040.0,VND,1241040.0
2,E4S4M3,TPAPI,1.0,1344472.0,2022-09-14 17:02:45,2022-09-15 00:02:45,2022-09-14 17:04:31,2022-09-15 00:04:31,tvlkapi03,NaN,...,NaN,NaN,NaN,VND,VND,614640.0,VND,614640.0,VND,614640.0
3,O5JRC1,TPAPI,1.0,1344473.0,2022-09-14 17:05:23,2022-09-15 00:05:23,2022-09-14 17:06:55,2022-09-15 00:06:55,tvlkapi03,NaN,...,NaN,NaN,NaN,VND,VND,614640.0,VND,614640.0,VND,614640.0
4,CITTEP,TPAPI,1.0,1344485.0,2022-09-14 17:02:41,2022-09-15 00:02:41,2022-09-14 17:06:56,2022-09-15 00:06:56,tvlkapi03,NaN,...,NaN,NaN,NaN,VND,VND,917040.0,VND,917040.0,VND,917040.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886,70EDEN,TPAPI,1.0,1346365.0,2022-09-15 16:35:53,2022-09-15 23:35:53,2022-09-15 16:57:22,2022-09-15 23:57:22,37120066,NaN,...,NaN,NaN,NaN,VND,VND,1230240.0,VND,1230240.0,VND,1230240.0
1887,GL751V,TPAPI,1.0,1346357.0,2022-09-15 16:56:31,2022-09-15 23:56:31,2022-09-15 16:57:59,2022-09-15 23:57:59,tvlkapi03,NaN,...,NaN,NaN,NaN,VND,VND,1111440.0,VND,1111440.0,VND,1111440.0
1888,13FYFC,TPAPI,1.0,1346358.0,2022-09-15 16:58:17,2022-09-15 23:58:17,2022-09-15 16:58:18,2022-09-15 23:58:18,AG120038S123,NaN,...,NaN,NaN,NaN,VND,VND,1111440.0,VND,1111440.0,VND,1111440.0
1889,E14XKX,TPAPI,1.0,1346366.0,2022-09-15 16:51:18,2022-09-15 23:51:18,2022-09-15 16:59:16,2022-09-15 23:59:16,tvlkapi04,NaN,...,NaN,NaN,NaN,VND,VND,1229280.0,VND,1229280.0,VND,1229280.0
